<a href="https://colab.research.google.com/github/dakilaledesma/VIGOR2/blob/main/notebooks/%5BNI%5D_Kinematics_to_Music_Motion_Visualizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! curl -L https://herbarium.ucht.cloud/s/AoZFbaKWofEfHiF/download --output motions.zip
! curl -L https://herbarium.ucht.cloud/s/Cn55KNbEBqtrDCg/download --output mp3.zip
! curl -L https://herbarium.ucht.cloud/s/MayD6rAGWoKmGw8/download --output keypoints3d.zip
! unzip -q motions.zip
! unzip -q keypoints3d.zip
! unzip -q mp3.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  305M  100  305M    0     0  37.6M      0  0:00:08  0:00:08 --:--:-- 49.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.6M  100 90.6M    0     0  27.4M      0  0:00:03  0:00:03 --:--:-- 27.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 72  834M   72  601M    0     0  44.6M      0  0:00:18  0:00:13  0:00:05 45.6M

In [ ]:
import pickle
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

output_str = ""
idx = 0
with open(glob("keypoints3d/*.pkl")[0], "rb") as motion:
  m = pickle.load(motion)

  for i in range(len(m["keypoints3d"])):
    m_np = m["keypoints3d"][i].flatten()
    print(m_np.shape)
    
    output_str += ','.join([str(round(v, 5)) for v in m_np])
    output_str += "\n"
    idx += 1

output_file = open("out.csv", 'w')
output_file.write(output_str)
output_file.close()

Streaming output truncated to the last 5000 lines.
 -9.60248658e+00  1.43444920e+02 -9.88180920e+01  1.05892115e+01
  1.10067791e+02 -7.93919764e+01 -9.94559772e+00  1.08989825e+02
 -9.03110225e+01  6.73112554e+00  7.57300953e+01 -8.65267424e+01
 -5.12392043e+00  7.48869556e+01 -9.78453145e+01]
(51,)
[-10.95471657 200.8673893  -75.27388146  -7.92291339 204.62649795
 -76.58464857 -14.76520659 203.37333296 -80.23535406   1.13829436
 205.87034714 -82.85682156 -11.78124666 204.26569488 -90.02160421
  13.92242911 190.24760689 -85.06852609 -13.97812092 190.84058744
 -99.82052769  20.760886   166.9566247  -85.69090555 -22.83385148
 172.11088691 -84.61933868  20.58717251 147.74633027 -86.1593866
 -21.73002199 180.13351912 -64.42537724   8.38079648 143.10456583
 -90.91838671  -9.9040069  143.22639893 -98.71684595  10.06016851
 110.17245573 -78.9197971  -10.9751185  108.76171876 -90.91827361
   6.30780185  75.72951679 -85.34059783  -5.34313505  74.60808032
 -98.36939575]
(51,)
[ -8.71647565 201.

In [ ]:
audio_dict = {}

In [ ]:
# Thanks to https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
import librosa
import os

lengths = []
for aud in glob("mp3/*.mp3"):
  my_audio_as_np_array, my_sample_rate = librosa.load(aud)

  spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                          sr=my_sample_rate, 
                                              n_fft=2048, 
                                              hop_length=512, 
                                              win_length=None, 
                                              window='hann', 
                                              center=True, 
                                              pad_mode='reflect', 
                                              power=2.0,
                                      n_mels=128)
  spec = spec[127, :1273]
  print(spec.shape)
  lengths.append(spec.shape[1]) # Everything is (128, -1)

print(min(lengths)) # Minimum is 1273

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


(1273,)


IndexError: ignored

In [ ]:
import pickle
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

lengths = []
for fn in glob("keypoints3d/*.pkl"):
  with open(fn, "rb") as motion:
    m = pickle.load(motion)

    lengths.append(m["keypoints3d"].shape[0]) # Shape is (-1, 17, 3), axis=0 is all different

print(min(lengths)) # Shortest is 426

426


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

input_img = keras.Input(shape=(426, 17, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
res = librosa.feature.inverse.mel_to_audio(spec, 
                                           sr=my_sample_rate, 
                                           n_fft=2048, 
                                           hop_length=512, 
                                           win_length=None, 
                                           window='hann', 
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)

import soundfile as sf
sf.write("test1.wav", res, sashi_sr)